In [35]:
import pandas as pd

In [36]:
from models import Line, Weather, Station, Train, Turnstile

In [37]:
curve_df = pd.read_csv("data/ridership_curve.csv")
curve_df.head(5)

,hour,ridership_ratio
0,0,0.0100
1,1,0.0100
2,2,0.0100
3,3,0.0100
4,4,0.0125


In [38]:
seed_df = pd.read_csv("data/ridership_seed.csv")
seed_df.head(5)

,station_id,stationame,month_beginning,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal
0,40380,Clark/Lake,10/01/2018,22811.6,6901.8,6233.5,577208
1,40260,State/Lake,10/01/2018,14098.7,7307.8,5824.5,376799
2,40200,Randolph/Wabash,10/01/2018,0.0,0.0,0.0,0
3,41700,Washington/Wabash,10/01/2018,11079.3,5920.0,4248.8,295498
4,40680,Adams/Wabash,10/01/2018,8448.0,3616.3,4136.0,225312


In [39]:
raw_df = pd.read_csv("data/cta_stations.csv").sort_values("order")

In [40]:
raw_df.head(5)

,stop_id,direction_id,stop_name,station_name,station_descriptive_name,station_id,order,red,blue,green
0,30004,W,Harlem (Terminal arrival),Harlem/Lake,Harlem/Lake (Green Line),40020,0,False,False,True
1,30003,E,Harlem (63rd-bound),Harlem/Lake,Harlem/Lake (Green Line),40020,0,False,False,True
81,30171,N,O'Hare (Terminal Arrival),O'Hare,O'Hare (Blue Line),40890,0,False,True,False
80,30172,S,O'Hare (Forest Pk-bound),O'Hare,O'Hare (Blue Line),40890,0,False,True,False
6,30173,N,Howard (Terminal arrival),Howard,"Howard (Red, Purple & Yellow Lines)",40900,0,True,False,False


In [41]:
station_df = raw_df[raw_df["blue"]]

In [42]:
color = Line.colors.blue

In [43]:
stations = station_df["station_name"].unique()

In [44]:
stations

array(["O'Hare", 'Rosemont', 'Cumberland', 'Harlem', 'Jefferson Park',
       'Montrose', 'Irving Park', 'Addison', 'Belmont', 'Logan Square',
       'California', "Western/O'Hare Branch", 'Damen', 'Division',
       'Chicago', 'Grand', 'Clark/Lake', 'Washington', 'Monroe',
       'Jackson', 'LaSalle', 'Clinton', 'UIC-Halsted', 'Racine',
       'Illinois Medical District', 'Western/Forest Pk Branch',
       'Kedzie-Homan', 'Pulaski', 'Cicero', 'Austin', 'Oak Park',
       'Forest Park'], dtype=object)

In [45]:
stations[0]

"O'Hare"

In [46]:
station_data = station_df[station_df["station_name"] == stations[0]]
station_data

,stop_id,direction_id,stop_name,station_name,station_descriptive_name,station_id,order,red,blue,green
81,30171,N,O'Hare (Terminal Arrival),O'Hare,O'Hare (Blue Line),40890,0,False,True,False
80,30172,S,O'Hare (Forest Pk-bound),O'Hare,O'Hare (Blue Line),40890,0,False,True,False


In [47]:
station_id = station_data["station_id"].unique()[0]
station_id

40890

In [48]:
station1 = Station(station_id, stations[0], color)

dict_keys(['_schemas', 'connect-configs', '_confluent-ksql-default__command_topic', 'com.udacity.streams.pages', 'connect-status', 'connect-offsets', '__consumer_offsets', 'com.udacity.streams.users', '_confluent-metrics', 'ohare_arrival', 'com.udacity.streams.clickevents', 'com.udacity.streams.purchases', '__confluent.support.metrics'])
dict_keys(['_schemas', 'connect-configs', '_confluent-ksql-default__command_topic', 'com.udacity.streams.pages', 'connect-status', 'connect-offsets', '__consumer_offsets', 'com.udacity.streams.users', '_confluent-metrics', 'ohare_arrival', 'com.udacity.streams.clickevents', 'com.udacity.streams.purchases', '__confluent.support.metrics'])
created new topic ohare_turnstile!!


In [49]:
station1.dir_a

In [50]:
train1 = Train("train1", Train.status.in_service)

In [51]:
train1.train_id

'train1'

In [52]:
train1.status

<status.in_service: 1>

In [53]:
print(train1)

Train ID train1 is in service


In [54]:
print(station1)

Station | 40890 | O'Hare                         | Direction A: |  ---  | departing to ---                            | Direction B: |  ---  | departing to ---                            | 


In [ ]:
stations = station_df["station_name"].unique()

station_data = station_df[station_df["station_name"] == stations[0]]
line = [
    Station(station_data["station_id"].unique()[0], stations[0], color)
]
prev_station = line[0]
for station in stations[1:]:
    station_data = station_df[station_df["station_name"] == station]
    new_station = Station(
        station_data["station_id"].unique()[0],
        station,
        color,
        prev_station,
    )
    prev_station.dir_b = new_station
    prev_station = new_station
    line.append(new_station)

In [59]:
line[0].dir_a, line[0].dir_b

(None,
 Station | 40820 | Rosemont                       | Direction A: |  ---  | departing to O'Hare                         | Direction B: |  ---  | departing to Cumberland                     | )

In [57]:
line[1].dir_a, line[1].dir_b

(Station | 40890 | O'Hare                         | Direction A: |  ---  | departing to ---                            | Direction B: |  ---  | departing to Rosemont                       | ,
 Station | 40230 | Cumberland                     | Direction A: |  ---  | departing to Rosemont                       | Direction B: |  ---  | departing to Harlem                         | )

In [58]:
turnstile1 = Turnstile(line[1])

dict_keys(['_schemas', 'connect-configs', '_confluent-ksql-default__command_topic', 'com.udacity.streams.pages', 'connect-status', 'connect-offsets', '__consumer_offsets', 'com.udacity.streams.users', '_confluent-metrics', 'ohare_arrival', 'com.udacity.streams.clickevents', 'com.udacity.streams.purchases', '__confluent.support.metrics'])
created new topic rosemont_turnstile!!


In [68]:
from datetime import datetime, timedelta

In [70]:
turnstile1.run(datetime(2020,10,10),timedelta(seconds=10))

AttributeError: 'int' object has no attribute 'encode'